#### Example 

In [ ]:
import pandas as pd
import requests

# Define parameters
country = "IN"
frequency = "A"
sector = "S1311B"
unit = "XDC"
indicator = "W0_S1_G21"
start_year = 2000
end_year = 2024

# Construct API URL
data_url = f"http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/GFSE/{frequency}.{country}.{sector}.{unit}.{indicator}?startPeriod={start_year}&endPeriod={end_year}"

# Fetch data
data_response = requests.get(data_url).json()

# Extract series data
series = data_response["CompactData"]["DataSet"]["Series"]
obs_list = series["Obs"]

# Define common metadata
common_data = {
    "Country Code": series["@REF_AREA"],
    "Frequency": series["@FREQ"],
    "Sector": series["@REF_SECTOR"],
    "Unit": series.get("@UNIT_MEASURE", ""),
    "Classification Indicator": series["@CLASSIFICATION"]
}

# Prepare data for DataFrame
rows = []
for obs in obs_list:
    year = obs["@TIME_PERIOD"]
    for key, value in obs.items():
        if key.startswith("@") and key != "@TIME_PERIOD":
            rows.append({**common_data, "Year": year, "Attribute": key[1:], "Value": value})

# Create DataFrame
df = pd.DataFrame(rows)


df.head(10)



##### Create a general function to get data from CompactData structure method 

In [ ]:
import pandas as pd
import requests
import os
import time

def fetch_imf_data(data_dict, data_folder, start_year, end_year, 
                   tracking_file="progress_tracking.csv", master_tracking_file="country_tracking.txt"):
    os.makedirs(data_folder, exist_ok=True)  

    if os.path.exists(tracking_file):
        progress_df = pd.read_csv(tracking_file)
        last_processed = {(row["Country Code"], row["Sector Code"], row["Unit Code"]): row["Last Classification Code"]
                          for _, row in progress_df.iterrows()}
    else:
        last_processed = {}

    master_records = []  
    request_count = 0  # To track API requests

    for country_code, country_name in data_dict['REF_AREA']['codelist'].items():
        country_data = []  

        for sector_code, sector_name in data_dict['REF_SECTOR']['codelist'].items():
            for unit_code, unit_name in data_dict['UNIT_MEASURE']['codelist'].items():
                key = (country_code, sector_code, unit_code)
                last_classification = last_processed.get(key, None)
                skip_flag = last_classification is not None

                for classification_code, classification_name in data_dict['CLASSIFICATION']['codelist'].items():
                    if skip_flag:
                        if classification_code == last_classification:
                            skip_flag = False  
                        continue  

                    url = f"http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/GFSE/A.{country_code}.{sector_code}.{unit_code}.{classification_code}?startPeriod={start_year}&endPeriod={end_year}"
                    
                    # Enforce rate limit
                    if request_count >= 10:
                        print("Rate limit reached. Waiting 2 seconds...")
                        time.sleep(2)
                        request_count = 0  # Reset counter after waiting
                    
                    try:
                        response = requests.get(url)
                        request_count += 1  # Increment API call counter

                        # Check if response is empty
                        if not response.text.strip():
                            print(f"Warning: Empty response for {country_code}-{sector_code}-{unit_code}-{classification_code}")
                            continue

                        data = response.json()
                        series = data.get("CompactData", {}).get("DataSet", {}).get("Series", {})

                        if not series or "Obs" not in series:
                            continue  
                        obs_list = series["Obs"]

                        common_data = {
                            "Country Code": country_code,
                            "Country Name": country_name,
                            "Sector Code": sector_code,
                            "Sector Name": sector_name,
                            "Unit Code": unit_code,
                            "Unit Name": unit_name,
                            "Classification Code": classification_code,
                            "Classification Name": classification_name
                        }
                        
                        for obs in obs_list:
                            if "@TIME_PERIOD" in obs:
                                year = obs["@TIME_PERIOD"]
                                for key, value in obs.items():
                                    if key.startswith("@") and key != "@TIME_PERIOD":
                                        row = {**common_data, "Year": year, "Attribute": key[1:], "Value": value}
                                        country_data.append(row)

                        last_processed[key] = classification_code  

                        progress_df = pd.DataFrame.from_dict([{
                            "Country Code": country_code,
                            "Sector Code": sector_code,
                            "Unit Code": unit_code,
                            "Last Classification Code": classification_code
                        }])

                        if os.path.exists(tracking_file):
                            existing_df = pd.read_csv(tracking_file)
                            existing_df = existing_df[existing_df["Country Code"] != country_code]
                            progress_df = pd.concat([existing_df, progress_df], ignore_index=True)

                        progress_df.to_csv(tracking_file, index=False)
                    except Exception as e:
                        print(f"Error fetching data for {country_code}-{sector_code}-{unit_code}-{classification_code}: {e}")
                        continue
        
        if country_data:
            df_country = pd.DataFrame(country_data)
            file_path = os.path.join(data_folder, f"{country_code}.csv")
            df_country.to_csv(file_path, index=False, mode="a", header=not os.path.exists(file_path))
            master_records.append(f"Country: {country_name} (Code: {country_code}), Data Saved: {file_path}")

    if master_records:
        with open(os.path.join(data_folder, master_tracking_file), "a") as f:
            f.write("\n".join(master_records) + "\n")
    
    print(f"Data processing complete for {start_year}-{end_year}. Files saved successfully.")


In [ ]:
data_dict_trial = {
    "FREQ": {
        "description": "Frequency",
        "codelist": None  # No codelist found for CL_FREQ
    },
    "REF_AREA": {
        "description": "Reference Area (Country or Region)",
        "codelist": {
          "US": "United States" , 
          "CL": "Chile",
          "AR": "Argentina"

        }
    },
    "REF_SECTOR": {
        "description": "Reference Sector",
        "codelist": {
            "S1311B": "Budgetary central government",
            "S1311": "Central government (excl. social security funds)",
            "S1321": "Central government (incl. social security funds)",
            "S13112": "Extrabudgetary central government",
            "S13": "General government",
            "S1313": "Local governments",
            "S1314": "Social security funds",
            "S1312": "State governments"
        }
    },
    "UNIT_MEASURE": {
        "description": "Unit of Measure",
        "codelist": {
            "XDC": "Domestic currency",
            "XDC_R_B1GQ": "Percent of GDP"
        }
    },
    "CLASSIFICATION": {
        "description": "Classification of Expense Items",
        "codelist": {
            "W0_S1_G2121": "Actual employers’ social contributions",
            "W0_S1_G21": "Compensation of employees",
            "W0_S1_G23": "Consumption of fixed capital",
            "W0_S1_G2811": "Dividend expense",
            "W0_S1_G212": "Employers' social contributions",
            "W0_S1_G273": "Employment-related social benefits expense",
            "W0_S1_G2": "Expense",
            "W0_S1_G2832": "Expense on NI & SGS: Capital claims",
            "W0_S1_G28313": "Expense on NI & SGS: Current claims",
            "W0_S1_G28312": "Expense on NI & SGS: Fees",
            "W0_S1_G28311": "Expense on NI & SGS: Premiums",
            "W0_S1_G283": "Expense on NI & SGS: Premiums, fees, & claims",
            "W0_S1_G2831": "Expense on NI & SGS: Premiums, fees, & current claims",
            "W0_S1_G282": "Expense on other transfers",
            "W00_S1W_G2822": "Expense on other transfers, capital",
            "W0_S1_G2821": "Expense on other transfers, current",
            "W0_S1_G26": "Grants expense",
            "W1_S13_G26": "Grants expense to foreign govts",
            "W1_S13_G26K": "Grants expense to foreign govts: capital",
            "W1_S13_G26C": "Grants expense to foreign govts: current",
            "1A_S1_G26": "Grants expense to int orgs",
            "1A_S1_G26K": "Grants expense to int orgs: capital",
            "1A_S1_G26C": "Grants expense to int orgs: current",
            "W2_S13_G26": "Grants expense to other gen govt",
            "W2_S13_G26K": "Grants expense to other gen govt: capital",
            "W2_S13_G26C": "Grants expense to other gen govt: current",
            "W0_S1_G2122": "Imputed employers’ social contributions",
            "W0_S1_G24": "Interest expense",
            "W1_S1_G24": "Interest expense to nonresidents",
            "W2_S13_G24": "Interest expense to other gen gov",
            "W2_S1W_G24": "Interest expense to residents other than gen govt",
            "W0_S1_G28": "Other expense",
            "W0_S1_G2813": "Property expense for investment income disbursements",
            "W0_S1_G281": "Property expense other than interest",
            "W0_S1_G2815": "Reinvested earnings on FDI",
            "W0_S1_G2814": "Rent expense",
            "W0_S1_G272": "Social assistance benefits expense",
            "W0_S1_G27": "Social benefits expense",
            "W0_S1_G2M27A": "Social benefits expense in cash",
            "W0_S1_G2M27B": "Social benefits expense in kind",
            "W0_S1_G271": "Social security benefits expense",
            "W0_S1_G25": "Subsidies expense",
            "W2_S1L_G25": "Subsidies expense to other sectors",
            "W2_S1KP_G25": "Subsidies expense to private enterprises",
            "W2_S1K1_G25": "Subsidies expense to public corporations",
            "W0_S1_G22": "Use of goods and services",
            "W0_S1_G211": "Wages and salaries",
            "W0_S1_G2812": "Withdrawals of income from quasi-corporations"
        }
    }
}


In [ ]:
data_folder = "Edit your file path here"

fetch_imf_data(data_dict_trial, data_folder, start_year=1990, end_year=2024)
